In [14]:
# tutorial found at: https://courses.engr.illinois.edu/ie534/fa2018/NLP.html#part-1-bag-of-words
from torch.autograd import Variable
from BOW_model import BOW_model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributed as dist

import time
import os
import sys
import io
import numpy as np
import os
import nltk
import itertools
import io
import tarfile


class BOW_model(nn.Module):
    def __init__(self, vocab_size, no_of_hidden_units):
        super(BOW_model, self).__init__()
        ## will need to define model architecture
        
    def forward(self, x, t):
        # will need to define forward function for when model gets called
        
    def __init__(self, vocab_size, no_of_hidden_units):
        super(BOW_model, self).__init__()

        self.embedding = nn.Embedding(vocab_size,no_of_hidden_units)

        self.fc_hidden = nn.Linear(no_of_hidden_units,no_of_hidden_units)
        self.bn_hidden = nn.BatchNorm1d(no_of_hidden_units)
        self.dropout = torch.nn.Dropout(p=0.5)

        self.fc_output = nn.Linear(no_of_hidden_units, 1)
        
        self.loss = nn.BCEWithLogitsLoss()
            
    def forward(self, x, t):
    
        bow_embedding = []
        for i in range(len(x)):
            lookup_tensor = Variable(torch.LongTensor(x[i])).cuda()
            embed = self.embedding(lookup_tensor)
            embed = embed.mean(dim=0)
            bow_embedding.append(embed)
        bow_embedding = torch.stack(bow_embedding)
    
        h = self.dropout(F.relu(self.bn_hidden(self.fc_hidden(bow_embedding))))
        h = self.fc_output(h)
    
        return self.loss(h[:,0],t), h[:,0]
    

In [ ]:
# pre processing data

#my_tar = tarfile.open('C:/Users/huang/Documents/pythonUTIR_NLP/UTIR-Sentiment-Analysis/aclImdb_v1.tar.gz')
#my_tar.extractall('./my_folder') # specify which folder to extract to
#my_tar.close()

## create directory to store preprocessed data
if(not os.path.isdir('preprocessed_data')):
    os.mkdir('preprocessed_data')
    
train_directory = 'C:/Users/huang/Documents/pythonUTIR_NLP/UTIR-Sentiment-Analysis/aclImdb/train'
pos_filenames = os.listdir(train_directory + '/pos/')

neg_filenames = os.listdir(train_directory + '/neg/')
unsup_filenames = os.listdir(train_directory + '/unsup/')

pos_filenames = [train_directory+'/pos/'+filename for filename in pos_filenames]
neg_filenames = [train_directory+'/neg/'+filename for filename in neg_filenames]
unsup_filenames = [train_directory+'/unsup/'+filename for filename in unsup_filenames]


In [ ]:
# train_directory = 'C:/Users/kaush/Desktop/aclImdb/train'
# pos_filenames = os.listdir(train_directory + '/pos/')

# neg_filenames = os.listdir(train_directory + '/neg/')
# unsup_filenames = os.listdir(train_directory + '/unsup/')

# pos_filenames = [train_directory+'/pos/'+filename for filename in pos_filenames]
# neg_filenames = [train_directory+'/neg/'+filename for filename in neg_filenames]
# unsup_filenames = [train_directory+'/unsup/'+filename for filename in unsup_filenames]

filenames = pos_filenames + neg_filenames + unsup_filenames

count = 0
x_train = []
for filename in filenames:
    with io.open(filename,'r',encoding='cp437') as f:
        line = f.readlines()[0]
    line = line.replace('<br />',' ')
    line = line.replace('\x96',' ')
    line = nltk.word_tokenize(line)
    line = [w.lower() for w in line]

    x_train.append(line)
    count += 1

In [ ]:
## get all of the test reviews
test_directory = 'C:/Users/huang/Documents/pythonUTIR_NLP/UTIR-Sentiment-Analysis/aclImdb/test'
#test_directory = 'C:/Users/kaush/Desktop/aclImdb/test'

pos_filenames = os.listdir(test_directory + '/pos/')
neg_filenames = os.listdir(test_directory + '/neg/')

pos_filenames = [test_directory+'/pos/'+filename for filename in pos_filenames]
neg_filenames = [test_directory+'/neg/'+filename for filename in neg_filenames]

filenames = pos_filenames+neg_filenames

count = 0
x_test = []
for filename in filenames:
    with io.open(filename,'r',encoding='utf-8') as f:
        line = f.readlines()[0]
    line = line.replace('<br />',' ')
    line = line.replace('\x96',' ')
    line = nltk.word_tokenize(line)
    line = [w.lower() for w in line]

    x_test.append(line)
    count += 1
    

In [6]:
## number of tokens per review
no_of_tokens = []
for tokens in x_train:
    no_of_tokens.append(len(tokens))
no_of_tokens = np.asarray(no_of_tokens)
print('Total: ', np.sum(no_of_tokens), ' Min: ', np.min(no_of_tokens), ' Max: ', np.max(no_of_tokens), ' Mean: ', np.mean(no_of_tokens), ' Std: ', np.std(no_of_tokens))

Total:  997807  Min:  26  Max:  2738  Mean:  273.7467764060357  Std:  210.76569615609978


In [ ]:
### word_to_id and id_to_word. associate an id to every unique token in the training data
all_tokens = itertools.chain.from_iterable(x_train)
word_to_id = {token: idx for idx, token in enumerate(set(all_tokens))}

all_tokens = itertools.chain.from_iterable(x_train)
id_to_word = [token for idx, token in enumerate(set(all_tokens))]
id_to_word = np.asarray(id_to_word)

In [9]:
## let's sort the indices by word frequency instead of random
x_train_token_ids = [[word_to_id[token] for token in x] for x in x_train]
count = np.zeros(id_to_word.shape)
for x in x_train_token_ids:
    for token in x:
        count[token] += 1
indices = np.argsort(-count)
id_to_word = id_to_word[indices]
count = count[indices]

hist = np.histogram(count,bins=[1,10,100,1000,10000]); #print(hist)
# for i in range(10):
#     print(id_to_word[i],count[i])

(array([160758,  26834,   7815,   1372], dtype=int64), array([    1,    10,   100,  1000, 10000]))
the 1009642.0
, 829575.0
. 817078.0
and 492150.0
a 488701.0
of 438552.0
to 405684.0
is 330975.0
it 285939.0
in 280663.0


In [ ]:
## recreate word_to_id based on sorted list
word_to_id = {token: idx for idx, token in enumerate(id_to_word)}

## assign -1 if token doesn't appear in our dictionary
## add +1 to all token ids, we went to reserve id=0 for an unknown token
x_train_token_ids = [[word_to_id.get(token,-1)+1 for token in x] for x in x_train]
x_test_token_ids = [[word_to_id.get(token,-1)+1 for token in x] for x in x_test]

In [ ]:
## save dictionary
np.save('preprocessed_data/imdb_dictionary.npy',np.asarray(id_to_word))

## save training data to single text file
with io.open('preprocessed_data/imdb_train.txt','w',encoding='utf-8') as f:
    for tokens in x_train_token_ids:
        for token in tokens:
            f.write("%i " % token)
        f.write("\n")

## save test data to single text file
with io.open('preprocessed_data/imdb_test.txt','w',encoding='utf-8') as f:
    for tokens in x_test_token_ids:
        for token in tokens:
            f.write("%i " % token)
        f.write("\n")

In [13]:
glove_filename = 'C:/Users/huang/Documents/pythonUTIR_NLP/UTIR-Sentiment-Analysis/glove.840B.300d.txt'
#glove_filename = 'C:/Users/kaush/Glove/glove.840B.300d.txt'

with io.open(glove_filename,'r',encoding='utf-8') as f:
    lines = f.readlines()

glove_dictionary = []
glove_embeddings = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split(' ')
    glove_dictionary.append(line[0])
    embedding = np.asarray(line[1:],dtype=np.float)
    glove_embeddings.append(embedding)
    count+=1
    if(count>=100000):
        break

glove_dictionary = np.asarray(glove_dictionary)
glove_embeddings = np.asarray(glove_embeddings)
# added a vector of zeros for the unknown tokens
glove_embeddings = np.concatenate((np.zeros((1,300)),glove_embeddings))

In [ ]:
word_to_id = {token: idx for idx, token in enumerate(glove_dictionary)}

x_train_token_ids = [[word_to_id.get(token,-1)+1 for token in x] for x in x_train]
x_test_token_ids = [[word_to_id.get(token,-1)+1 for token in x] for x in x_test]

In [ ]:
np.save('preprocessed_data/glove_dictionary.npy',glove_dictionary)
np.save('preprocessed_data/glove_embeddings.npy',glove_embeddings)

with io.open('preprocessed_data/imdb_train_glove.txt','w',encoding='utf-8') as f:
    for tokens in x_train_token_ids:
        for token in tokens:
            f.write("%i " % token)
        f.write("\n")

with io.open('preprocessed_data/imdb_test_glove.txt','w',encoding='utf-8') as f:
    for tokens in x_test_token_ids:
        for token in tokens:
            f.write("%i " % token)
        f.write("\n")

In [ ]:
# 'bow model'